In [1]:
import math
import os
import sys

import numpy as np
from PIL import Image

/Users/sbroome/miniconda3/envs/temporal_shape/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [2]:
def arr_from_img(im, mean=0, std=1):
    '''
    Args:
        im: Image
        shift: Mean to subtract
        std: Standard Deviation to subtract
    Returns:
        Image in np.float32 format, in width height channel format. With values in range 0,1
        Shift means subtract by certain value. Could be used for mean subtraction.
    '''
    width, height = im.size
    arr = im.getdata()
    c = int(np.product(arr.size) / (width * height))

    return (np.asarray(arr, dtype=np.float32).reshape((height, width, c)).transpose(2, 1, 0) / 255. - mean) / std


def get_image_from_array(X, index, mean=0, std=1):
    '''
    Args:
        X: Dataset of shape N x C x W x H
        index: Index of image we want to fetch
        mean: Mean to add
        std: Standard Deviation to add
    Returns:
        Image with dimensions H x W x C or H x W if it's a single channel image
    '''
    ch, w, h = X.shape[1], X.shape[2], X.shape[3]
    ret = (((X[index] + mean) * 255.) * std).reshape(ch, w, h).transpose(2, 1, 0).clip(0, 255).astype(np.uint8)
    if ch == 1:
        ret = ret.reshape(h, w)
    return ret

# loads mnist from web on demand
def load_dataset(training=True):
    if sys.version_info[0] == 2:
        from urllib import urlretrieve
    else:
        from urllib.request import urlretrieve

    def download(filename, source='http://yann.lecun.com/exdb/mnist/'):
        print("Downloading %s" % filename)
        urlretrieve(source + filename, filename)

    import gzip
    def load_mnist_images(filename):
        if not os.path.exists(filename):
            download(filename)
        with gzip.open(filename, 'rb') as f:
            data = np.frombuffer(f.read(), np.uint8, offset=16)
        data = data.reshape(-1, 1, 28, 28).transpose(0, 1, 3, 2)
        return data / np.float32(255)

    if training:
        return load_mnist_images('train-images-idx3-ubyte.gz')
    return load_mnist_images('t10k-images-idx3-ubyte.gz')

/Users/sbroome/miniconda3/envs/temporal_shape/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
def generate_moving_mnist(training, shape=(64, 64), num_frames=30, num_images=2, original_size=28, nums_per_image=3, traj_per_image=2):
    '''
    Args:
        training: Boolean, used to decide if downloading/generating train set or test set
        shape: Shape we want for our moving images (new_width and new_height)
        num_frames: Number of frames in a particular movement/animation/gif
        num_images: Number of movement/animations/gif to generate
        original_size: Real size of the images (eg: MNIST is 28x28)
        nums_per_image: Digits per movement/animation/gif.
    Returns:
        Dataset of np.uint8 type with dimensions num_frames * num_images x 1 x new_width x new_height
    '''
    mnist = load_dataset(training)
    width, height = shape

    # Get how many pixels can we move around a single image (to fit its width)
    lims = (x_lim, y_lim) = width - original_size, height - original_size
    print(lims)

    # Create a dataset of shape of num_frames * num_images x 1 x new_width x new_height
    # Eg : 3000000 x 1 x 64 x 64
    dataset = np.empty((num_frames * num_images, 1, width, height), dtype=np.uint8)

    for img_idx in range(num_images):
        # Randomly generate direction, speed and velocity for both images
        direcs = np.pi * (np.random.rand(traj_per_image) * 2 - 1)  # Scalars, one per digit
        direcs = np.insert(direcs, 0, direcs[0])
        speeds = np.random.randint(5, size=traj_per_image) + 2  # Scalars, one per digit
        speeds = np.insert(speeds, 0, speeds[0])
        # veloc is 2xnums_per_image (x and y component for velocity for each digit)
        veloc = np.asarray([(speed * math.cos(direc), speed * math.sin(direc)) for direc, speed in zip(direcs, speeds)])
        
        # Get a list containing three PIL images randomly sampled from the database
        mnist_images = [Image.fromarray(get_image_from_array(mnist, r, mean=0)).resize((original_size, original_size),
                                                                                       Image.ANTIALIAS) \
                        for r in np.random.randint(0, mnist.shape[0], nums_per_image)]
        # Generate tuples of (x,y) i.e initial positions for nums_per_image (default : 2)
        import pdb; pdb.set_trace()
        positions = np.asarray([(np.random.rand() * x_lim, np.random.rand() * y_lim) for _ in range(nums_per_image)])
        print(positions)

        # Generate new frames for the entire num_framesgth
        for frame_idx in range(num_frames):

            canvases = [Image.new('L', (width, height)) for _ in range(nums_per_image)]
            canvas = np.zeros((1, width, height), dtype=np.float32)

            # In canv (i.e Image object) place the image at the respective positions
            # Super impose both images on the canvas (i.e empty np array)
            for i, canv in enumerate(canvases):
                canv.paste(mnist_images[i], tuple(positions[i].astype(int)))
                canvas += arr_from_img(canv, mean=0)

            # Get the next position by adding velocity
            next_pos = positions + veloc

            # Iterate over velocity and see if we hit the wall
            # If we do then change the  (change direction)
            for i, pos in enumerate(next_pos):
                for j, coord in enumerate(pos):
                    if coord < -2 or coord > lims[j] + 2:
                        veloc[i] = list(list(veloc[i][:j]) + [-1 * veloc[i][j]] + list(veloc[i][j + 1:]))

            # Make the permanent change to position by adding updated velocity
            positions = positions + veloc

            # Add the canvas to the dataset array
            dataset[img_idx * num_frames + frame_idx] = (canvas * 255).clip(0, 255).astype(np.uint8)

    return dataset

In [6]:
def main(training, dest, filetype='jpg', frame_size=64, num_frames=30, num_images=2, original_size=28,
         nums_per_image=3):
    dat = generate_moving_mnist(training, shape=(frame_size, frame_size), num_frames=num_frames, num_images=num_images, \
                                original_size=original_size, nums_per_image=nums_per_image)
    n = num_images * num_frames
    if filetype == 'npz':
        np.savez(dest, dat)
    elif filetype == 'jpg':
        for i in range(dat.shape[0]):
            Image.fromarray(get_image_from_array(dat, i, mean=0)).save(os.path.join(dest, '{}.jpg'.format(i)))

In [8]:
training = False
dest = 'movingmnist/'
main(training, dest)

In [10]:
training = False
dest = 'mm_3digits_test'
main(training, dest)

In [6]:
training = False
dest = 'mm_fixedtraj'
main(training, dest, num_images=2)

> <ipython-input-5-5d248565afe1>(26)generate_moving_mnist()
     24         # Randomly generate direction, speed and velocity for both images
     25         import pdb; pdb.set_trace()
---> 26         direcs = np.pi * (np.random.rand(nums_per_image) * 2 - 1)
     27         speeds = np.random.randint(5, size=nums_per_image) + 2
     28         veloc = np.asarray([(speed * math.cos(direc), speed * math.sin(direc)) for direc, speed in zip(direcs, speeds)])



ipdb>  n


> <ipython-input-5-5d248565afe1>(27)generate_moving_mnist()
     25         import pdb; pdb.set_trace()
     26         direcs = np.pi * (np.random.rand(nums_per_image) * 2 - 1)
---> 27         speeds = np.random.randint(5, size=nums_per_image) + 2
     28         veloc = np.asarray([(speed * math.cos(direc), speed * math.sin(direc)) for direc, speed in zip(direcs, speeds)])
     29         # Get a list containing two PIL images randomly sampled from the database



ipdb>  direcs


array([-1.61019798,  2.66725922, -1.23545881])


...

ipdb>  direcs


array([-1.61019798,  2.66725922, -1.23545881])


ipdb>  c


> <ipython-input-5-5d248565afe1>(25)generate_moving_mnist()
     23     for img_idx in range(num_images):
     24         # Randomly generate direction, speed and velocity for both images
---> 25         import pdb; pdb.set_trace()
     26         direcs = np.pi * (np.random.rand(nums_per_image) * 2 - 1)
     27         speeds = np.random.randint(5, size=nums_per_image) + 2



ipdb>  c


In [ ]:
training = False
dest = 'mm_fixedtrajj'
main(training, dest, num_images=5)

(36, 36)
> <ipython-input-5-471732ca1cb1>(39)generate_moving_mnist()
     37         # Generate tuples of (x,y) i.e initial positions for nums_per_image (default : 2)
     38         import pdb; pdb.set_trace()
---> 39         positions = np.asarray([(np.random.rand() * x_lim, np.random.rand() * y_lim) for _ in range(nums_per_image)])
     40         print(positions)
     41 



In [8]:
training = False
dest = 'mm_fixedtraj'
main(training, dest, num_images=2)

> <ipython-input-7-637f9ca470e5>(26)generate_moving_mnist()
     24         # Randomly generate direction, speed and velocity for both images
     25         import pdb; pdb.set_trace()
---> 26         direcs = np.pi * (np.random.rand(traj_per_image) * 2 - 1)  # Scalars, one per digit
     27         direcs = np.insert(direcs, 0, direcs[0])
     28         speeds = np.random.randint(5, size=traj_per_image) + 2  # Scalars, one per digit



ipdb>  c


> <ipython-input-7-637f9ca470e5>(25)generate_moving_mnist()
     23     for img_idx in range(num_images):
     24         # Randomly generate direction, speed and velocity for both images
---> 25         import pdb; pdb.set_trace()
     26         direcs = np.pi * (np.random.rand(traj_per_image) * 2 - 1)  # Scalars, one per digit
     27         direcs = np.insert(direcs, 0, direcs[0])



ipdb>  c


In [5]:
training = False
dest = 'mm_fixedtraj'
main(training, dest, num_images=5)

(36, 36)
> <ipython-input-3-8ffef408d90c>(39)generate_moving_mnist()
     37         # Generate tuples of (x,y) i.e initial positions for nums_per_image (default : 2)
     38         import pdb; pdb.set_trace()
---> 39         positions = np.asarray([(np.random.rand() * x_lim, np.random.rand() * y_lim) for _ in range(nums_per_image)])
     40         print(positions)
     41 



ipdb>  canvas.shape


*** NameError: name 'canvas' is not defined


ipdb>  n


> <ipython-input-3-8ffef408d90c>(40)generate_moving_mnist()
     38         import pdb; pdb.set_trace()
     39         positions = np.asarray([(np.random.rand() * x_lim, np.random.rand() * y_lim) for _ in range(nums_per_image)])
---> 40         print(positions)
     41 
     42         # Generate new frames for the entire num_framesgth



...

[[18.38389138 23.76249601]
 [12.83637761  4.83036468]
 [27.29794718  9.51264482]]
> <ipython-input-3-8ffef408d90c>(38)generate_moving_mnist()
     36                         for r in np.random.randint(0, mnist.shape[0], nums_per_image)]
     37         # Generate tuples of (x,y) i.e initial positions for nums_per_image (default : 2)
---> 38         import pdb; pdb.set_trace()
     39         positions = np.asarray([(np.random.rand() * x_lim, np.random.rand() * y_lim) for _ in range(nums_per_image)])
     40         print(positions)



ipdb>  c


[[10.29785606 34.6154634 ]
 [27.92868252 32.04371026]
 [35.83528446  2.98893907]]
> <ipython-input-3-8ffef408d90c>(39)generate_moving_mnist()
     37         # Generate tuples of (x,y) i.e initial positions for nums_per_image (default : 2)
     38         import pdb; pdb.set_trace()
---> 39         positions = np.asarray([(np.random.rand() * x_lim, np.random.rand() * y_lim) for _ in range(nums_per_image)])
     40         print(positions)
     41 



ipdb>  c


[[25.75683875  0.35709424]
 [ 7.45169274 23.16003736]
 [31.19650222 28.64120006]]
